In [1]:
from models.todo import TodoSchema, Todo
import json
from pymongo import MongoClient

In [65]:

class MongoDAO:
    def __init__(self, collection, schema):
        self._db = collection
        self._schema = schema

    def get_all(self):
        todos_dict = list(self._db.find())
        for todo in todos_dict:
            todo['_id'] = str(todo['_id'])
        return self._schema(many=True).load(todos_dict)

    def get_by_uuid(self, uuid):
        todo_dict = self._db.find_one({'uuid':uuid})
        if todo_dict:
            todo_dict['_id'] = str(todo_dict['_id'])
            return self._schema().load(todo_dict)

    def add_item(self, item):
        item_dict = self._schema().dump(item)
        r = self._db.insert_one(item_dict)
        return r.inserted_id

    def delete_by_uuid(self, uuid):
        return self._db.delete_one({'uuid':uuid}).deleted_count

    def update_by_uuid(self, uuid, data):
        if not self._db.find_one({'uuid':uuid}):
            return False
        return self._db.update_one({'uuid':uuid},{'$set':data}).acknowledged
    
    


In [3]:
import yaml
from lib.common import make_url, replace_env

In [4]:
with open('/config/todos/default_config.yml','r') as f:
    config = yaml.load(f, yaml.SafeLoader)
replace_env(config)

{'database': {'mongo': {'type': 'mongodb',
   'host': 'mongo',
   'user': 'mongouser',
   'password': 'averysecurepassword',
   'port': '27017',
   'db': 'todos_db'}}}

In [48]:
todo = TodoSchema().load({
    'user_id': 0,
    'title':'Something to do.'
})

In [6]:
todo

Todo(user_id=0, uuid='TODO-5b7123a1-e952-4c90-a699-fa4d02c2d3df', title='Something to do.', completed=False)

In [7]:
url = make_url(config['database']['mongo'],include_db=False)

In [8]:
client = MongoClient(url)
collection = client.todos.todos_collection

In [62]:
dao = MongoDAO(collection,TodoSchema)

In [51]:
dao.add_item(todo)

ObjectId('5e14c4798637a2d727d71819')

In [66]:
dao.get_all()

[Todo(user_id=0, uuid='TODO-37dcbb23-1741-4cd2-a91f-4c0b9297e700', title='A new original title!', completed=False)]

In [21]:
id_str = str(collection.find_one()['_id'])

In [22]:
from bson import ObjectId

In [23]:
ObjectId(id_str) == collection.find_one()['_id']

True

In [24]:
dao.get_by_uuid('TODO-5b7123a1-e952-4c90-a699-fa4d02c2d3df')

Todo(user_id=0, uuid='TODO-5b7123a1-e952-4c90-a699-fa4d02c2d3df', title='Something to do.', completed=False)

In [44]:
dao.delete_by_uuid('TODO-f943a3b6-096b-44be-b08b-a13ad2ef6b52')

0

In [64]:
dao.update_by_uuid('TODO-37dcbb23-1741-4cd2-a91f-4c0b9 297e700',{'title': 'A new original title!'})

In [67]:
class MyObject:
    pass

In [68]:
a = MyObject

In [69]:
a.something = 'here'

In [71]:
a.something

'here'

In [73]:
d = {}

In [78]:
d['something'] = 'here'

In [96]:
def my_function(mutate, default=config_factory("/config/todos/default_config.yml")):
    if mutate:
        default['something'] = 'here'
    return default

called


In [97]:
my_function(False)

{'database': {'mongo': {'type': 'mongodb',
   'host': 'mongo',
   'user': 'mongouser',
   'password': 'averysecurepassword',
   'port': '27017',
   'db': 'todos_db'}}}

In [95]:
def config_factory(path):
    print('called')
    with open(path, "r") as f:
        config = yaml.load(f, yaml.SafeLoader)
    replace_env(config)
    return config

{'database': {'mongo': {'type': 'mongodb',
   'host': 'mongo',
   'user': 'mongouser',
   'password': 'averysecurepassword',
   'port': '27017',
   'db': 'todos_db'}}}